In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import os
from SDMBench import *

In [2]:
path = '/home/yzy/projects/SDMBench/analysis/unzip/default_comparison'

In [3]:
data_dict = {
    '151507':'Dataset1@Visium@Sequencing',
    '151508':'Dataset2@Visium@Sequencing',
    '151509':'Dataset3@Visium@Sequencing',
    '151510':'Dataset4@Visium@Sequencing',
    '151669':'Dataset5@Visium@Sequencing',
    '151670':'Dataset6@Visium@Sequencing',
    '151671':'Dataset7@Visium@Sequencing',
    '151672':'Dataset8@Visium@Sequencing',
    '151673':'Dataset9@Visium@Sequencing',
    '151674':'Dataset10@Visium@Sequencing',
    '151675':'Dataset11@Visium@Sequencing',
    '151676':'Dataset12@Visium@Sequencing',
    
    'global_z0':'Dataset13@SMI@Sequencing',
    'global_z1':'Dataset14@SMI@Sequencing',
    'global_z2':'Dataset15@SMI@Sequencing',
    'global_z3':'Dataset16@SMI@Sequencing',
    'global_z4':'Dataset17@SMI@Sequencing',
    
    'Slice_1':'Dataset18@BaristaSeq@Sequencing',
    'Slice_2':'Dataset19@BaristaSeq@Sequencing',
    'Slice_3':'Dataset20@BaristaSeq@Sequencing',
    
    'mouse_VISp':'Dataset21@SeqFISH@Imaging',
    
    'MERFISH_0.04':'Dataset22@MERFISH@Imaging',
    'MERFISH_0.09':'Dataset23@MERFISH@Imaging',
    'MERFISH_0.14':'Dataset24@MERFISH@Imaging',
    'MERFISH_0.19':'Dataset25@MERFISH@Imaging',
    'MERFISH_0.24':'Dataset26@MERFISH@Imaging',
    
    'osmfish':'Dataset27@osmFISH@Imaging',
    
    '20180417_BZ5_control':'Dataset28@STARmap@Imaging',
    '20180419_BZ9_control':'Dataset29@STARmap@Imaging',
    '20180424_BZ14_control':'Dataset30@STARmap@Imaging',
    
    'STARmap_20180505_BY3_1k':'Dataset31@STARmap1k@Imaging'
        
}

data_gt_dict = {
    '151507':'Region',
    '151508':'Region',
    '151509':'Region',
    '151510':'Region',
    '151669':'Region',
    '151670':'Region',
    '151671':'Region',
    '151672':'Region',
    '151673':'Region',
    '151674':'Region',
    '151675':'Region',
    '151676':'Region',
    
    'global_z0':'region',
    'global_z1':'region',
    'global_z2':'region',
    'global_z3':'region',
    'global_z4':'region',
    
    'Slice_1':'layer',
    'Slice_2':'layer',
    'Slice_3':'layer',
    
    'mouse_VISp':'layer',
    
    'MERFISH_0.04':'Region',
    'MERFISH_0.09':'Region',
    'MERFISH_0.14':'Region',
    'MERFISH_0.19':'Region',
    'MERFISH_0.24':'Region',
    
    'osmfish':'Region',
    
    '20180417_BZ5_control':'region',
    '20180419_BZ9_control':'region',
    '20180424_BZ14_control':'region',
    
    'STARmap_20180505_BY3_1k':'region'
        
}

In [4]:
pd_dict = {
    'DataName':[],
    'DataID':[],
    'Biotech':[],
    'BiotechType':[],
    'Replicate':[],
    'Method':[],
    'NMI':[],
    'HOM':[],
    'COM':[],
    'CHAOS':[],
    'PAS':[],
    'ASW':[],
    'MoranI':[],
    'GearyC':[],
    'time':[],
    'memory':[]
    
}

In [5]:
def update_pd(pd_dict,DataName,DataID,Biotech,BiotechType,Replicate,Method,NMI,HOM,COM,CHAOS,PAS,ASW,MoranI,GearyC,time,memory):
    
    pd_dict['DataName'].append(DataName)
    pd_dict['DataID'].append(DataID)    
    pd_dict['Biotech'].append(Biotech)    
    pd_dict['BiotechType'].append(BiotechType)    
    pd_dict['Replicate'].append(Replicate)    
    pd_dict['Method'].append(Method)    
    pd_dict['NMI'].append(NMI)    
    pd_dict['HOM'].append(HOM)    
    pd_dict['COM'].append(COM)    
    pd_dict['CHAOS'].append(CHAOS)    
    pd_dict['PAS'].append(PAS)    
    pd_dict['ASW'].append(ASW)    
    pd_dict['MoranI'].append(MoranI)    
    pd_dict['GearyC'].append(GearyC)    
    pd_dict['time'].append(time) 
    pd_dict['memory'].append(memory)        

In [12]:
import collections
X_dict = collections.defaultdict(list)
import os
rootdir = path

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        file_whole = os.path.join(subdir, file)


        if file.endswith('.h5ad'):
            
            methodname = subdir.split('/')[-1]
            dataname_idx2 = file.rfind('.')
            dataname_idx1 = file.rfind(methodname)+len(methodname)
            dataname = file[dataname_idx1+1:dataname_idx2]
            
            
            for dn in data_dict.keys():
                if dn in dataname:
                    dataname = dn
                    break
            
            if f'{dataname}@{methodname}' in uni:
                print(f'{dataname}@{methodname}','exist')
                continue
            
            adata = sc.read_h5ad(file_whole)
            gt_key = data_gt_dict[dataname]
            adata = adata[np.logical_not(adata.obs[gt_key].isna())]
            
            DatasetID = data_dict[dataname].split('@')[0]
            Biotech = data_dict[dataname].split('@')[1]
            BiotechType = data_dict[dataname].split('@')[2]
            print(methodname,dataname,gt_key in adata.obs)
            
            for i in range(10):
                Replicate = i
                adata.obs[f'pred_{i+1}'] = adata.obs[f'pred_{i+1}'].astype('str')
                adata.obs[f'pred_{i+1}'] = adata.obs[f'pred_{i+1}'].astype('category')
                
                adata.obs[gt_key] = adata.obs[gt_key].astype('str')
                adata.obs[gt_key] = adata.obs[gt_key].astype('category')
                
                if np.unique(adata.obs[f'pred_{i+1}']).shape[0]==1:
                    continue
                
                NMI = compute_NMI(adata,gt_key,f'pred_{i+1}')
                COM = compute_COM(adata,gt_key,f'pred_{i+1}')
                HOM = compute_HOM(adata,gt_key,f'pred_{i+1}')
                
                CHAOS = compute_CHAOS(adata,f'pred_{i+1}')
                PAS = compute_PAS(adata,f'pred_{i+1}')
                ASW = compute_ASW(adata,f'pred_{i+1}')
                
                MoranI,GearyC = marker_score(adata,f'pred_{i+1}',top_n=5)
                
                time = adata.uns['time'][i]
                memory = adata.uns['memory'][i]
                
                update_pd(pd_dict,dataname,DatasetID,Biotech,BiotechType,Replicate,methodname,NMI,HOM,COM,CHAOS,PAS,ASW,MoranI,GearyC,time,memory)
                
                
                

   
                
                


151671@SpaGCN_with exist
151676@SpaGCN_with exist
151508@SpaGCN_with exist
151509@SpaGCN_with exist
151674@SpaGCN_with exist
151510@SpaGCN_with exist
151507@SpaGCN_with exist
151675@SpaGCN_with exist
151672@SpaGCN_with exist
151673@SpaGCN_with exist
151670@SpaGCN_with exist
151669@SpaGCN_with exist
151670@SpaGCN_without exist
151671@SpaGCN_without exist
151676@SpaGCN_without exist
151508@SpaGCN_without exist
151509@SpaGCN_without exist
151674@SpaGCN_without exist
151510@SpaGCN_without exist
151507@SpaGCN_without exist
151675@SpaGCN_without exist
151672@SpaGCN_without exist
151669@SpaGCN_without exist
151673@SpaGCN_without exist
151671@SEDR exist
151670@SEDR exist
151509@SEDR exist
151676@SEDR exist
151508@SEDR exist
151507@SEDR exist
151675@SEDR exist
151674@SEDR exist
151510@SEDR exist
151669@SEDR exist
151672@SEDR exist
151673@SEDR exist
151674@BASS exist
151510@BASS exist
151507@BASS exist
151675@BASS exist
151672@BASS exist
151669@BASS exist
151670@BASS exist
151671@BASS exist
1515

/tmp/ipykernel_87640/4204923910.py:39: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[f'pred_{i+1}'] = adata.obs[f'pred_{i+1}'].astype('str')


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/tqdm-4.64.1-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.43/s]


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.37/s]


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 18.23/s]


...

/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.38/s]


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.80/s]


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.94/s]


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 11.08/s]


/home/yzy/anaconda3/envs/SDMBench/lib/python3.9/site-packages/scanpy-1.9.1-py3.9.egg/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.12/s]


In [14]:
1+1

2

In [13]:
pd_df = pd.DataFrame(pd_dict)
pd_df.to_feather('performance_summary_ext.feather')

In [2]:
pd_df_sub = pd.read_feather('performance_summary.feather')

In [19]:
pd_df = pd_df.append(pd_df_sub)

/tmp/ipykernel_87640/3178985264.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd_df = pd_df.append(pd_df_sub)


In [21]:
pd_df = pd_df.reset_index()

In [23]:
pd_df.to_feather('performance_summary.feather')

In [6]:
pd_df_sub[pd_df_sub['Biotech']=='Visium'].groupby('Method').agg('median')

/tmp/ipykernel_903/687360317.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pd_df_sub[pd_df_sub['Biotech']=='Visium'].groupby('Method').agg('median')


,index,Replicate,NMI,HOM,COM,CHAOS,PAS,ASW,MoranI,GearyC,time,memory
Method,,,,,,,,,,,,
BASS,419.5,4.5,0.611997,0.637169,0.601635,0.059851,0.028604,0.077486,0.336070,0.663803,777.357900,2107.000000
BayesSpace,1255.5,4.5,0.597890,0.615036,0.585416,0.060460,0.031447,0.062957,0.314897,0.686485,2387.427000,8315.100000
CCST,1135.5,4.5,0.509539,0.564786,0.480069,0.059767,0.010775,0.118043,0.303576,0.697273,255.204987,1724.443315
DeepST,1615.5,4.5,0.621003,0.642188,0.605053,0.060922,0.092262,0.043793,0.309660,0.690933,473.626454,3437.004033
SCAN-IT,895.5,4.5,0.598712,0.638695,0.561996,0.059761,0.011131,0.100115,0.313041,0.686662,4926.581189,1438.499743
SEDR,299.5,4.5,0.533891,0.445146,0.693166,0.060684,0.014724,0.072285,0.376058,0.624559,341.212129,975.258046
STAGATE,1375.5,4.5,0.494056,0.513094,0.477734,0.061015,0.081392,0.039719,0.336325,0.663230,18.799392,334.690156
SpaGCN_with,59.5,4.5,0.482514,0.499250,0.470395,0.064560,0.243146,0.009688,0.319458,0.678853,94.155574,2163.110340
SpaGCN_without,179.5,4.5,0.522473,0.541607,0.505602,0.062843,0.131224,0.065760,0.313786,0.687942,94.626476,1134.751388
